<img src="https://storage.googleapis.com/novo-blog-wordpress/2023/07/853ab090-capa-blog_projeto-orientacao-profissional_engenharia-civil.jpg">

### <strong><em><snap style=color:navy> Frameworks

In [5]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StringIndexer, MinMaxScaler
from pyspark.ml.stat import Correlation
from pyspark.ml.regression import *
from pyspark.ml.evaluation import *
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [6]:
%reload_ext watermark
%watermark -a "Claudio Sturaro Martinez Junior" --iversions

Author: Claudio Sturaro Martinez Junior

findspark: 2.0.1
sys      : 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]
pyspark  : 3.5.0
numpy    : 1.24.3
decimal  : 1.70



### <strong><em><snap style=color:navy> Preparando o Ambiente Spark

In [ ]:
import findspark
findspark.init()

In [7]:
sc = SparkContext(appName='Mini-Project-5')

In [8]:
sc.setLogLevel('ERROR')

In [22]:
spark = SparkSession.builder.master('local').getOrCreate()

In [23]:
spark

### <strong><em><snap style=color:navy> Carregando o Dataset

In [25]:
# carregar dados  como dataframe do spark	
dados = spark.read.csv(r"C:\FCD\Python_Analise_de_Dados\ML_Mini_Projeto_5\dados\dataset.csv", inferSchema=True, header=True)

In [26]:
# conferindo o tipo do dataset
type(dados)

pyspark.sql.dataframe.DataFrame

In [29]:
# quantas linhas tem o df
dados.count()

1030

In [28]:
# visualizacao dos dados df do spark
dados.show()

+------+-----+------+-----+----------------+---------------+-------------+---+-----+
|cement| slag|flyash|water|superplasticizer|coarseaggregate|fineaggregate|age|csMPa|
+------+-----+------+-----+----------------+---------------+-------------+---+-----+
| 540.0|  0.0|   0.0|162.0|             2.5|         1040.0|        676.0| 28|79.99|
| 540.0|  0.0|   0.0|162.0|             2.5|         1055.0|        676.0| 28|61.89|
| 332.5|142.5|   0.0|228.0|             0.0|          932.0|        594.0|270|40.27|
| 332.5|142.5|   0.0|228.0|             0.0|          932.0|        594.0|365|41.05|
| 198.6|132.4|   0.0|192.0|             0.0|          978.4|        825.5|360| 44.3|
| 266.0|114.0|   0.0|228.0|             0.0|          932.0|        670.0| 90|47.03|
| 380.0| 95.0|   0.0|228.0|             0.0|          932.0|        594.0|365| 43.7|
| 380.0| 95.0|   0.0|228.0|             0.0|          932.0|        594.0| 28|36.45|
| 266.0|114.0|   0.0|228.0|             0.0|          932.0|     

In [31]:
# visualizacao dos dados df pandas
dados.toPandas()

,cement,slag,flyash,water,superplasticizer,coarseaggregate,fineaggregate,age,csMPa
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30
...,...,...,...,...,...,...,...,...,...
1025,276.4,116.0,90.3,179.6,8.9,870.1,768.3,28,44.28
1026,322.2,0.0,115.6,196.0,10.4,817.9,813.4,28,31.18
1027,148.5,139.4,108.6,192.7,6.1,892.4,780.0,28,23.70
1028,159.1,186.7,0.0,175.6,11.3,989.6,788.9,28,32.77


In [34]:
# schema
dados.printSchema()

root
 |-- cement: double (nullable = true)
 |-- slag: double (nullable = true)
 |-- flyash: double (nullable = true)
 |-- water: double (nullable = true)
 |-- superplasticizer: double (nullable = true)
 |-- coarseaggregate: double (nullable = true)
 |-- fineaggregate: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- csMPa: double (nullable = true)



### <strong><em><snap style=color:navy> Modulo de Automacao da Preparacao de Dados

O MLlib requer que todas as colunas de entrada do dataframe sejam vetorizadas. Vamos criar um funcao Python que ira automatizar nosso trabalho de preparacao dos dados, incluindo a vetorizacao e todas as tarefas necessarias.

In [35]:
# separamos os dados ausentes (se existirem) e removemos (se existirem)
dados_com_linhas_removidas = dados.na.drop()
print('numero de linhas do dataframe original:', dados.count())
print('numero de linhas do novo dataframe:', dados_com_linhas_removidas.count())

numero de linhas do dataframe original: 1030
numero de linhas do novo dataframe: 1030


In [59]:
# Função de preparação dos dados
def func_modulo_prep_dados(df,
                           variaveis_entrada,
                           variavel_saida,
                           tratar_outliers = True,
                           padronizar_dados = True):

    print("Iniciando o modulo de preparacao dos dados")
	# Vamos gerar um novo dataframe, renomeando o argumento que representa a variável de saída.
    novo_df = df.withColumnRenamed(variavel_saida, 'label')
    
	# Convertemos a variável alvo para o tipo numérico como float (encoding)
    if str(novo_df.schema['label'].dataType) != 'IntegerType':
        novo_df = novo_df.withColumn("label", novo_df["label"].cast(FloatType()))
        
    # Listas de controle para as variáveis
    variaveis_numericas = []
    variaveis_categoricas = []
    
    # Se tiver variáveis de entrada do tipo string, convertemos para o tipo numérico
    for coluna in variaveis_entrada:
        
        # Verifica se a variável é do tipo string
        if str(novo_df.schema[coluna].dataType) == 'StringType':
            
            # Definimos a variável com um sufixo
            novo_nome_coluna = coluna + "_num"
            
            # Adicionamos à lista de variáveis categóricas
            variaveis_categoricas.append(novo_nome_coluna)
            
        else:
            
            # Se não for variável do tipo string, então é numérica e adicionamos na lista correspondente
            variaveis_numericas.append(coluna)
            
            # Colocamos os dados no dataframe de variáveis indexadas
            df_indexed = novo_df
            
    # Se o dataframe tiver dados do tipo string, aplicamos a indexação
    # Verificamos se a lista de variáveis categóricas não está vazia
    if len(variaveis_categoricas) != 0: 
        
        # Loop pelas colunas
        for coluna in novo_df:
            
            # Se a variável é do tipo string, criamos, treinamos e aplicamos o indexador
            if str(novo_df.schema[coluna].dataType) == 'StringType':
                
                # Cria o indexador
                indexer = StringIndexer(inputCol = coluna, outputCol = coluna + "_num") 
                
                # Treina e aplica o indexador
                df_indexed = indexer.fit(novo_df).transform(novo_df)
    else:
        
        # Se não temos mais variáveis categóricas, então colocamos os dados no dataframe de variáveis indexadas
        df_indexed = novo_df
        
    # Se for necessário tratar outliers, faremos isso agora
    if tratar_outliers == True:
        print("\nAplicando o tratamento de outliers...")
        
        # Dicionário
        d = {}
        
        # Dicionário de quartis das variáveis do dataframe indexado (somente variáveis numéricas)
        for col in variaveis_numericas: 
            d[col] = df_indexed.approxQuantile(col,[0.01, 0.99], 0.25) 
        
        # Agora aplicamos transformação dependendo da distribuição de cada variável
        for col in variaveis_numericas:
            
            # Extraímos a assimetria dos dados e usamos isso para tratar os outliers
            skew = df_indexed.agg(skewness(df_indexed[col])).collect() 
            skew = skew[0][0]
            
            # Verificamos a assimetria e então aplicamos:
            
            # Transformação de log + 1 se a assimetria for positiva
            if skew > 1:
                indexed = df_indexed.withColumn(col, log(when(df[col] < d[col][0], d[col][0])\
                .when(df_indexed[col] > d[col][1], d[col][1])\
                .otherwise(df_indexed[col] ) + 1).alias(col))
                print("\nA variável " + col + " foi tratada para assimetria positiva (direita) com skew =", skew)
            
            # Transformação exponencial se a assimetria for negativa
            elif skew < -1:
                indexed = df_indexed.withColumn(col, \
                exp(when(df[col] < d[col][0], d[col][0])\
                .when(df_indexed[col] > d[col][1], d[col][1])\
                .otherwise(df_indexed[col] )).alias(col))
                print("\nA variável " + col + " foi tratada para assimetria negativa (esquerda) com skew =", skew)
                
            # Assimetria entre -1 e 1 não precisamos aplicar transformação aos dados

    # Vetorização
    
    # Lista final de atributos
    lista_atributos = variaveis_numericas + variaveis_categoricas
    
    # Cria o vetorizador para os atributos
    vetorizador = VectorAssembler(inputCols = lista_atributos, outputCol = 'features')
    
    # Aplica o vetorizador ao conjunto de dados
    dados_vetorizados = vetorizador.transform(df_indexed).select('features', 'label')
    
    # Se a flag padronizar_dados está como True, então padronizamos os dados colocando-os na mesma escala
    if padronizar_dados == True:
        print("\nPadronizando o conjunto de dados para o intervalo de 0 a 1...")
        
        # Cria o scaler
        scaler = MinMaxScaler(inputCol = "features", outputCol = "scaledFeatures")

        # Calcula o sumário de estatísticas e gera o padronizador
        global scalerModel
        scalerModel = scaler.fit(dados_vetorizados)

        # Padroniza as variáveis para o intervalo [min, max]
        dados_padronizados = scalerModel.transform(dados_vetorizados)
        
        # Gera os dados finais
        dados_finais = dados_padronizados.select('label', 'scaledFeatures')
        
        # Renomeia as colunas (requerido pelo Spark)
        dados_finais = dados_finais.withColumnRenamed('scaledFeatures', 'features')
        
        print("\nProcesso Concluído!")

    # Se a flag está como False, então não padronizamos os dados
    else:
        print("\nOs dados não serão padronizados pois a flag padronizar_dados está com o valor False.")
        dados_finais = dados_vetorizados
    
    return dados_finais

Agora aplicamos o modulo de preparacao dos dados.

In [60]:
# lista de variaveis de entrada (todas menos a target)
variaveis_entrada = dados.columns[:-1]

In [61]:
# variavel alvo / target
variavel_saida = dados.columns[-1]

In [62]:
# aplica a funcao
dados_finais = func_modulo_prep_dados(df=dados,
									  variaveis_entrada=variaveis_entrada,
									  variavel_saida=variavel_saida)

Iniciando o modulo de preparacao dos dados

Aplicando o tratamento de outliers...

A variável age foi tratada para assimetria positiva (direita) com skew = 3.2644145354168086

Padronizando o conjunto de dados para o intervalo de 0 a 1...

Processo Concluído!


In [64]:
# visualiza os dados
dados_finais.show(truncate=False)

+-----+------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                      |
+-----+------------------------------------------------------------------------------------------------------------------------------+
|79.99|[1.0,0.0,0.0,0.3210862619808307,0.07763975155279502,0.6947674418604651,0.20572002007024587,0.07417582417582418]               |
|61.89|[1.0,0.0,0.0,0.3210862619808307,0.07763975155279502,0.7383720930232558,0.20572002007024587,0.07417582417582418]               |
|40.27|[0.526255707762557,0.3964941569282137,0.0,0.8482428115015974,0.0,0.3808139534883721,0.0,0.739010989010989]                    |
|41.05|[0.526255707762557,0.3964941569282137,0.0,0.8482428115015974,0.0,0.3808139534883721,0.0,1.0]                                  |
|44.3 |[0.22054794520547943,0.3683917640511965,0.0,0.56

### <strong><em><snap style=color:navy> Verificando a Correlacao

Vamos nos certificar que nao temos multicolinearidade antes de prosseguirmos.
Lembre-se das seguintes diretrizes para o Coeficiente de Correlacao de Pearson:

* .00 - .19 (correlacao muito franca)
* .20 - .39 (correlacao fraca)
* .40 - .59 (correlacao moderada)
* .60 - .79 (correlacao forte)
* .80 - 1.0 (correlacao muito forte)


<img src="https://dhg1h5j42swfq.cloudfront.net/2021/12/19151647/image-411.png">

In [65]:
# extrai a correlacao
coeficientes_corr = Correlation.corr(dados_finais, 'features', 'pearson').collect()[0][0]
array_corr = coeficientes_corr.toArray()

In [66]:
array_corr

array([[ 1.        , -0.27521591, -0.39746734, -0.08158675,  0.09238617,
        -0.10934899, -0.22271785,  0.08194602],
       [-0.27521591,  1.        , -0.3235799 ,  0.10725203,  0.04327042,
        -0.28399861, -0.28160267, -0.04424602],
       [-0.39746734, -0.3235799 ,  1.        , -0.25698402,  0.37750315,
        -0.00996083,  0.07910849, -0.15437052],
       [-0.08158675,  0.10725203, -0.25698402,  1.        , -0.65753291,
        -0.1822936 , -0.45066117,  0.27761822],
       [ 0.09238617,  0.04327042,  0.37750315, -0.65753291,  1.        ,
        -0.26599915,  0.22269123, -0.19270003],
       [-0.10934899, -0.28399861, -0.00996083, -0.1822936 , -0.26599915,
         1.        , -0.17848096, -0.00301588],
       [-0.22271785, -0.28160267,  0.07910849, -0.45066117,  0.22269123,
        -0.17848096,  1.        , -0.1560947 ],
       [ 0.08194602, -0.04424602, -0.15437052,  0.27761822, -0.19270003,
        -0.00301588, -0.1560947 ,  1.        ]])

In [67]:
# lista de correlacao entre os atributos e a variavel alvo
for item in array_corr:
	print(item[7])

0.08194602387182176
-0.044246019304454175
-0.15437051606792915
0.27761822152100296
-0.19270002804347258
-0.0030158803467436645
-0.15609470264758615
1.0


### <strong><em><snap style=color:navy> Divisao de Dados de Treino e Teste

In [68]:
dados_treino, dados_teste = dados_finais.randomSplit([.7, .3])

In [69]:
dados_treino.count()

724

In [71]:
dados_teste.count()

306

### <strong><em><snap style=color:navy> Modulo de AutoML (Automated Machine Learning)

In [72]:
# Módulo de Machine Learning
def func_modulo_ml(algoritmo_regressao):

    # Função para obter o tipo do algoritmo de regressão e criar a instância do objeto
    # Usaremos isso para automatizar nosso processo
    def func_tipo_algo(algo_regressao):
        algoritmo = algo_regressao
        tipo_algo = type(algoritmo).__name__
        return tipo_algo
    
    # Aplica a função anterior
    tipo_algo = func_tipo_algo(algoritmo_regressao)

    # Se o algoritmo for Regressão Linear, entramos neste bloco if
    if tipo_algo == "LinearRegression":
        
        # Treinamos a primeira versão do modelo sem validação cruzada
        modelo = regressor.fit(dados_treino)
        
        # Métricas do modelo
        print('\033[1m' + "Modelo de Regressão Linear Sem Validação Cruzada:" + '\033[0m')
        print("")
        
        # Avalia o modelo com dados de teste
        resultado_teste = modelo.evaluate(dados_teste)

        # Imprime as métricas de erro do modelo com dados de teste
        print("RMSE em Teste: {}".format(resultado_teste.rootMeanSquaredError))
        print("Coeficiente R2 em Teste: {}".format(resultado_teste.r2))
        print("")
        
        # Agora vamos criar a segunda versão do modelo com mesmo algoritmo, mas usando validação cruzada
        
        # Prepara o grid de hiperparâmetros
        paramGrid = (ParamGridBuilder().addGrid(regressor.regParam, [0.1, 0.01]).build())
        
        # Cria os avaliadores
        eval_rmse = RegressionEvaluator(metricName = "rmse")
        eval_r2 = RegressionEvaluator(metricName = "r2")
        
        # Cria o Cross Validator
        crossval = CrossValidator(estimator = regressor,
                                  estimatorParamMaps = paramGrid,
                                  evaluator = eval_rmse,
                                  numFolds = 3) 
        
        print('\033[1m' + "Modelo de Regressão Linear Com Validação Cruzada:" + '\033[0m')
        print("")
        
        # Treina o modelo com validação cruzada
        modelo = crossval.fit(dados_treino)
        
        # Salva o melhor modelo da versão 2
        global LR_BestModel 
        LR_BestModel = modelo.bestModel
                
        # Previsões com dados de teste
        previsoes = LR_BestModel.transform(dados_teste)
        
        # Avaliação do melhor modelo
        resultado_teste_rmse = eval_rmse.evaluate(previsoes)
        print('RMSE em Teste:', resultado_teste_rmse)
        
        resultado_teste_r2 = eval_r2.evaluate(previsoes)
        print('Coeficiente R2 em Teste:', resultado_teste_r2)
        print("")
    
        # Lista de colunas para colocar no dataframe de resumo
        columns = ['Regressor', 'Resultado_RMSE', 'Resultado_R2']
        
        # Formata os resultados e cria o dataframe
        
        # Formata as métricas e nome do algoritmo
        rmse_str = [str(resultado_teste_rmse)] 
        r2_str = [str(resultado_teste_r2)] 
        tipo_algo = [tipo_algo] 
        
        # Cria o dataframne
        df_resultado = spark.createDataFrame(zip(tipo_algo, rmse_str, r2_str), schema = columns)
        
        # Grava os resultados no dataframe
        df_resultado = df_resultado.withColumn('Resultado_RMSE', df_resultado.Resultado_RMSE.substr(0, 5))
        df_resultado = df_resultado.withColumn('Resultado_R2', df_resultado.Resultado_R2.substr(0, 5))
        
        return df_resultado

    else:
        
        # Verificamos se o algoritmo é o Decision Tree e criamos o grid de hiperparâmetros
        if tipo_algo in("DecisionTreeRegressor"):
            paramGrid = (ParamGridBuilder().addGrid(regressor.maxBins, [10, 20, 40]).build())

        # Verificamos se o algoritmo é o Random Forest e criamos o grid de hiperparâmetros
        if tipo_algo in("RandomForestRegressor"):
            paramGrid = (ParamGridBuilder().addGrid(regressor.numTrees, [5, 20]).build())

        # Verificamos se o algoritmo é o GBT e criamos o grid de hiperparâmetros
        if tipo_algo in("GBTRegressor"):
            paramGrid = (ParamGridBuilder() \
                         .addGrid(regressor.maxBins, [10, 20]) \
                         .addGrid(regressor.maxIter, [10, 15])
                         .build())
            
        # Verificamos se o algoritmo é o Isotonic 
        if tipo_algo in("IsotonicRegression"):
            paramGrid = (ParamGridBuilder().addGrid(regressor.isotonic, [True, False]).build())

        # Cria os avaliadores
        eval_rmse = RegressionEvaluator(metricName = "rmse")
        eval_r2 = RegressionEvaluator(metricName = "r2")
        
        # Prepara o Cross Validator
        crossval = CrossValidator(estimator = regressor,
                                  estimatorParamMaps = paramGrid,
                                  evaluator = eval_rmse,
                                  numFolds = 3) 
        
        # Treina o modelo usando validação cruzada
        modelo = crossval.fit(dados_treino)
        
        # Extrai o melhor modelo
        BestModel = modelo.bestModel

        # Resumo de cada modelo
        
        # Métricas do modelo
        if tipo_algo in("DecisionTreeRegressor"):
            
            # Variável global
            global DT_BestModel 
            DT_BestModel = modelo.bestModel
            
            # Previsões com dados de teste
            previsoes_DT = DT_BestModel.transform(dados_teste)
            
            print('\033[1m' + "Modelo Decision Tree Com Validação Cruzada:" + '\033[0m')
            print(" ")
            
            # Avaliação do modelo
            resultado_teste_rmse = eval_rmse.evaluate(previsoes_DT)
            print('RMSE em Teste:', resultado_teste_rmse)
        
            resultado_teste_r2 = eval_r2.evaluate(previsoes_DT)
            print('Coeficiente R2 em Teste:', resultado_teste_r2)
            print("")
        
        # Métricas do modelo
        if tipo_algo in("RandomForestRegressor"):
            
            # Variável global
            global RF_BestModel 
            RF_BestModel = modelo.bestModel
            
            # Previsões com dados de teste
            previsoes_RF = RF_BestModel.transform(dados_teste)
            
            print('\033[1m' + "Modelo RandomForest Com Validação Cruzada:" + '\033[0m')
            print(" ")
            
            # Avaliação do modelo
            resultado_teste_rmse = eval_rmse.evaluate(previsoes_RF)
            print('RMSE em Teste:', resultado_teste_rmse)
        
            resultado_teste_r2 = eval_r2.evaluate(previsoes_RF)
            print('Coeficiente R2 em Teste:', resultado_teste_r2)
            print("")
        
        # Métricas do modelo
        if tipo_algo in("GBTRegressor"):

            # Variável global
            global GBT_BestModel 
            GBT_BestModel = modelo.bestModel
            
            # Previsões com dados de teste
            previsoes_GBT = GBT_BestModel.transform(dados_teste)
            
            print('\033[1m' + "Modelo Gradient-Boosted Tree (GBT) Com Validação Cruzada:" + '\033[0m')
            print(" ")
            
            # Avaliação do modelo
            resultado_teste_rmse = eval_rmse.evaluate(previsoes_GBT)
            print('RMSE em Teste:', resultado_teste_rmse)
        
            resultado_teste_r2 = eval_r2.evaluate(previsoes_GBT)
            print('Coeficiente R2 em Teste:', resultado_teste_r2)
            print("")
            
        # Métricas do modelo
        if tipo_algo in("IsotonicRegression"):

            # Variável global
            global ISO_BestModel 
            ISO_BestModel = modelo.bestModel
            
            # Previsões com dados de teste
            previsoes_ISO = ISO_BestModel.transform(dados_teste)
            
            print('\033[1m' + "Modelo Isotonic Com Validação Cruzada:" + '\033[0m')
            print(" ")
            
            # Avaliação do modelo
            resultado_teste_rmse = eval_rmse.evaluate(previsoes_ISO)
            print('RMSE em Teste:', resultado_teste_rmse)
        
            resultado_teste_r2 = eval_r2.evaluate(previsoes_ISO)
            print('Coeficiente R2 em Teste:', resultado_teste_r2)
            print("")
                    
        # Lista de colunas para colocar no dataframe de resumo
        columns = ['Regressor', 'Resultado_RMSE', 'Resultado_R2']
        
        # Faz previsões com dados de teste
        previsoes = modelo.transform(dados_teste)
        
        # Avalia o modelo para gravar o resultado
        eval_rmse = RegressionEvaluator(metricName = "rmse")
        rmse = eval_rmse.evaluate(previsoes)
        rmse_str = [str(rmse)]
        
        eval_r2 = RegressionEvaluator(metricName = "r2")
        r2 = eval_r2.evaluate(previsoes)
        r2_str = [str(r2)]
         
        tipo_algo = [tipo_algo] 
        
        # Cria o dataframe
        df_resultado = spark.createDataFrame(zip(tipo_algo, rmse_str, r2_str), schema = columns)
        
        # Grava o resultado no dataframe
        df_resultado = df_resultado.withColumn('Resultado_RMSE', df_resultado.Resultado_RMSE.substr(0, 5))
        df_resultado = df_resultado.withColumn('Resultado_R2', df_resultado.Resultado_R2.substr(0, 5))
        
        return df_resultado

In [73]:
# Lista de algoritmos
regressores = [LinearRegression(),
               DecisionTreeRegressor(),
               RandomForestRegressor(),
               GBTRegressor(),
               IsotonicRegression()] 

In [74]:
# Lista de colunas e valores
colunas = ['Regressor', 'Resultado_RMSE', 'Resultado_R2']
valores = [("N/A", "N/A", "N/A")]

In [75]:
# Prepara a tabela de resumo
df_resultados_treinamento = spark.createDataFrame(valores, colunas)

In [76]:
# Loop de treinamento
for regressor in regressores:
    
    # Para cada regressor obtém o resultado
    resultado_modelo = func_modulo_ml(regressor)
    
    # Grava os resultados
    df_resultados_treinamento = df_resultados_treinamento.union(resultado_modelo)

Modelo de Regressão Linear Sem Validação Cruzada:

RMSE em Teste: 10.524027720507188
Coeficiente R2 em Teste: 0.5563120597984652

Modelo de Regressão Linear Com Validação Cruzada:

RMSE em Teste: 10.52392326872791
Coeficiente R2 em Teste: 0.5563208670281772

Modelo Decision Tree Com Validação Cruzada:
 
RMSE em Teste: 8.069473343362006
Coeficiente R2 em Teste: 0.7391420720525737

Modelo RandomForest Com Validação Cruzada:
 
RMSE em Teste: 6.748382068514965
Coeficiente R2 em Teste: 0.8175629743143521

Modelo Gradient-Boosted Tree (GBT) Com Validação Cruzada:
 
RMSE em Teste: 6.18694330354034
Coeficiente R2 em Teste: 0.8466563028036085

Modelo Isotonic Com Validação Cruzada:
 
RMSE em Teste: 13.507357139323611
Coeficiente R2 em Teste: 0.2691059548967488



In [77]:
# Retorna as linhas diferentes de N/A
df_resultados_treinamento = df_resultados_treinamento.where("Regressor!='N/A'")

In [78]:
# Imprime
df_resultados_treinamento.show(10, False)

+---------------------+--------------+------------+
|Regressor            |Resultado_RMSE|Resultado_R2|
+---------------------+--------------+------------+
|LinearRegression     |10.52         |0.556       |
|DecisionTreeRegressor|8.069         |0.739       |
|RandomForestRegressor|6.748         |0.817       |
|GBTRegressor         |6.186         |0.846       |
|IsotonicRegression   |13.50         |0.269       |
+---------------------+--------------+------------+



## Conclusao:
O modelo GBTRegressor apresentou a melhor performance, pois o erro RMSE foi o menor (bom) e a precisao do R2 (compreensao da variabilidade dos dados) foi a maior (bom).